In [7]:
### Regressions Ridge et Lasso avec glmnet
### 25 septembre 2019
### Jeremy L Hour

library("MASS")
library("glmnet")

# load user-defined functions
source("functions/DataSim.R") 

# 1. Simulate data
data = DataSim(n=1000, p=20, Ry= .2, Intercept=F)
y = data$y; X = data$X

Warning message in c * gamma:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in c * b:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”

0.2490257
1.3140788
-0.5799046
-0.9818655
-1.3778077
-2.8981510
